# Setup

In [ ]:
## Python packages
import os, sys, stat, shutil
import pandas as pd
import numpy as np

In [ ]:
## Move to main directory
# Replace with your own path
os.chdir('/Users/CDarnell/Dropbox (Duke Bio_EA)/Schmid Lab/Cindy/Data/ChIP-seq/20170817_Hv_rosRHA')

In [ ]:
## Make directory tree

if not 'data_processing' in os.listdir('.'):
    os.mkdir('data_processing')
    os.mkdir('data_processing/genome')
    os.mkdir('data_processing/raw_data')
    os.mkdir('data_processing/trimmed_data')
    os.mkdir('data_processing/aligned_data')
    os.mkdir('data_processing/aligned_data/wiggle_files')
    os.mkdir('data_processing/aligned_data/GC_bias')
    os.mkdir('data_processing/peak_analysis')
    os.mkdir('data_processing/peak_analysis/MOSAiCS_results')
    os.mkdir('data_processing/peak_analysis/MOSAiCS_results/bins')
    os.mkdir('data_processing/peak_analysis/MOSAiCS_results/output')
    os.mkdir('data_processing/peak_analysis/MOSAiCS_results/plots')
    
os.chdir('./data_processing')

# Download and process reference genome

In [ ]:
# Download genome information from NCBI

! rsync --copy-links --recursive --times --verbose rsync://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/Haloferax_volcanii/all_assembly_versions/GCF_000025685.1_ASM2568v1 ./genome/


In [ ]:
# Make genome files writable

! chmod a+rw ./genome/GCF_000025685.1_ASM2568v1/*.gz
! chmod a+rw ./genome/GCF_000025685.1_ASM2568v1/*.txt


In [ ]:
# Rename sequence file and decompress

shutil.copy2('./genome/GCF_000025685.1_ASM2568v1/GCF_000025685.1_ASM2568v1_genomic.fna.gz', './genome/DS2.fna.gz')
shutil.copy2('./genome/GCF_000025685.1_ASM2568v1/GCF_000025685.1_ASM2568v1_genomic.gff.gz', './genome/DS2.gff.gz')

! gunzip ./genome/*.gz

In [ ]:
# Convert .fna to .2bit (for GC correction)

! faToTwoBit ./genome/DS2.fna ./genome/DS2.2bit

In [ ]:
# Build genome index library for Bowtie2

! bowtie2-build ./genome/DS2.fna ./genome/DS2

In [ ]:
# Create annotation bed file (for peak annotations using bedtools)

! sortBed -i ./genome/DS2.gff > ./genome/DS2_annotations.bed

# Processing fastq.gz data files

In [ ]:
## Download data files into ~/data_processing/raw_data
# These will be available from GEO

# This assumes the original files are backed up on a server/cloud.
# Rename and decompress

os.rename('./raw_data/CD1_S8_L007_R1_001.fastq.gz', './raw_data/pyrE2_WCE.fastq.gz')
os.rename('./raw_data/CD2_S9_L007_R1_001.fastq.gz', './raw_data/pyrE2_IP.fastq.gz')
os.rename('./raw_data/CD3_S10_L007_R1_001.fastq.gz', './raw_data/rosRHA1_WCE.fastq.gz')
os.rename('./raw_data/CD4_S11_L007_R1_001.fastq.gz', './raw_data/rosRHA1_IP.fastq.gz')
os.rename('./raw_data/CD5_S12_L007_R1_001.fastq.gz', './raw_data/rosRHA2_WCE.fastq.gz')
os.rename('./raw_data/CD6_S13_L007_R1_001.fastq.gz', './raw_data/rosRHA2_IP.fastq.gz')
os.rename('./raw_data/CD7_S14_L007_R1_001.fastq.gz', './raw_data/rosRHA3_WCE.fastq.gz')
os.rename('./raw_data/CD8_S15_L007_R1_001.fastq.gz', './raw_data/rosRHA3_IP.fastq.gz')
os.rename('./raw_data/CD9_S16_L007_R1_001.fastq.gz', './raw_data/rosRHA4_WCE.fastq.gz')
os.rename('./raw_data/CD10_S17_L007_R1_001.fastq.gz', './raw_data/rosRHA4_IP.fastq.gz')


! gunzip ./raw_data/*.fastq.gz

In [ ]:
! ls ./raw_data

In [ ]:
## Assess quality of reads

! fastqc -q ./raw_data/*.fastq

In [ ]:
! open ./raw_data/*.html

In [ ]:
## Trim adapter sequences

! trim_galore ./raw_data/*.fastq -o ./trimmed_data

In [ ]:
%%bash

## Align files with Bowtie2

cd ./trimmed_data
for file in *_trimmed.fq; do
bowtie2 -x ../genome/DS2 -U $file -S ../aligned_data/`basename $file .fq`.sam
done

In [ ]:
## Rename, because I don't know how to code off the "trimmed"

%cd ./aligned_data

os.rename('pyrE2_WCE_trimmed.sam', 'pyrE2_WCE.sam')
os.rename('pyrE2_IP_trimmed.sam', 'pyrE2_IP.sam')
os.rename('rosRHA1_WCE_trimmed.sam', 'rosRHA1_WCE.sam')
os.rename('rosRHA1_IP_trimmed.sam', 'rosRHA1_IP.sam')
os.rename('rosRHA2_WCE_trimmed.sam', 'rosRHA2_WCE.sam')
os.rename('rosRHA2_IP_trimmed.sam', 'rosRHA2_IP.sam')
os.rename('rosRHA3_WCE_trimmed.sam', 'rosRHA3_WCE.sam')
os.rename('rosRHA3_IP_trimmed.sam', 'rosRHA3_IP.sam')
os.rename('rosRHA4_WCE_trimmed.sam', 'rosRHA4_WCE.sam')
os.rename('rosRHA4_IP_trimmed.sam', 'rosRHA4_IP.sam')


# Alignment file processing

In [ ]:
%%bash

## Alignment processing
# Convert sam to bam file

for file in *.sam; do
samtools view -bS $file > `basename $file .sam`.bam;
done

In [ ]:
%%bash

# Sort bam files

for file in *.bam; do
samtools sort $file -o `basename $file .bam`_sorted.bam;
done

In [ ]:
%%bash

# Index bam files

for file in *_sorted.bam; do
samtools index $file `basename $file .bam`.bam.bai;
done

# GC Bias

In [ ]:
%%bash

## GC Bias
# Compute GC bias

for file in *_WCE_sorted.bam
do
computeGCBias -b $file --effectiveGenomeSize 4012900 -g ../genome/DS2.2bit -l 300 -freq ./GC_bias/`basename $file .bam`_GC.txt;
done

In [ ]:
%cd ..

## Generate (better) plots in R
Run GC_Bias_plots.R
You may need to make changes depending on your data/genome. 

These plots will need to be manually inspected to determine if you need to apply GC correction. See computeGCBias documentation at [deeptools](https://deeptools.readthedocs.io/en/develop/content/tools/computeGCBias.html)


## Correct GC Bias

In [ ]:
%%bash

cd ./aligned_data

for file in *_WCE_sorted.bam; do
computeGCBias -b $file --effectiveGenomeSize 4012900 -g ../genome/DS2.2bit -freq ./GC_bias/`basename $file .bam`_GC.txt -o ./GC_bias/`basename $file .bam`_GCcorrected.bam;
done


## Generate wiggle files with MOSAiCS

For data visualization purposes.
Run Generate_wiggle_files.R

For details, see documentation of [MOSAiCS package](https://bioconductor.org/packages/release/bioc/html/mosaics.html)

# Peak calling with MOSAiCS

Run Peak_calling.R

For details, see documentation of [MOSAiCS package](https://bioconductor.org/packages/release/bioc/html/mosaics.html)

# Calling gene annotations using bedtools

In [ ]:
# Copy annotations file to peak_analysis directory

! cp ./genome/ATCC33960_annotations.bed ./peak_analysis_MOSAiCS_results/output/ATCC33960_annotations.bed
%cd ./peak_analysis/MOSAiCS_results/output

# Compare biological replicates

! bedtools intersect -a pyrF1_minus_GC.bed -b pyrF2_minus_GC.bed -f 0.50 -r > all_pyrF_minus.bed
! bedtools intersect -a trmBHA1_plus_GC.bed -b trmBHA2_plus_GC.bed -f 0.50 -r > all_trmBHA_plus.bed
! bedtools multiinter -i trmBHA1_minus_GC.bed trmBHA2_minus_GC.bed trmBHA3_minus_GC.bed trmBHA4_minus_GC.bed > all_trmBHA_minus.bed


In [ ]:
# Subtract negative control peaks

! bedtools subtract -a all_trmBHA_minus.bed -b all_pyrF_minus.bed -f 0.50 -r > all_trmBHA_minus_subtracted.bed
! bedtools subtract -a all_trmBHA_plus.bed -b all_pyrF_minus.bed -f 0.50 -r > all_trmBHA_plus_subtracted.bed


In [ ]:
# Sort by chromosome

! bedtools sort -i all_trmBHA_minus_subtracted.bed > all_trmBHA_minus_subtracted_sorted.bed
! bedtools sort -i all_trmBHA_plus_subtracted.bed > all_trmBHA_plus_subtracted_sorted.bed


In [ ]:
# Pull closest annotations

! bedtools closest -a all_trmBHA_minus_subtracted_sorted.bed -b ATCC33960_annotations.bed > all_trmBHA_minus_closest.txt
! bedtools closest -a all_trmBHA_plus_subtracted_sorted.bed -b ATCC33960_annotations.bed > all_trmBHA_plus_closest.txt


In [ ]:
# Remove duplicates

! sort all_trmBHA_minus_closest.txt | uniq > all_trmBHA_minus_closest_noDuplicates.txt
! sort all_trmBHA_plus_closest.txt | uniq > all_trmBHA_plus_closest_noDuplicates.txt

! open all_trmBHA_minus_closest_noDuplicates.txt
! open all_trmBHA_plus_closest_noDuplicates.txt
